In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from ipywidgets import *
from PIL import Image
from IPython.display import display, clear_output
import io

import pytesseract
from pytesseract import Output as py_Output

In [2]:
MAX_FEATURES = 1000
GOOD_MATCH_PERCENT = 0.5

def alignImages(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    matches = list(matches)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
#     cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg 

In [3]:
btn1_upload = FileUpload(description='Ref. Img')
btn2_upload = FileUpload(description='Comp. Img')
out_pl_ref = Output()
out_pl_im = Output()
btn_load = Button(description='Show Images')

rotation = -90

def on_click_load(change):
    imReference = Image.open(io.BytesIO(btn1_upload.data[-1]))
    im = Image.open(io.BytesIO(btn2_upload.data[-1]))
    out_pl_ref.clear_output()
    out_pl_im.clear_output()
    out_pl_diff.clear_output()
    with out_pl_ref:
        imReference.thumbnail([450,450])
        display(imReference.rotate(rotation))
    with out_pl_im:
        im.thumbnail([450,450])
        display(im.rotate(rotation))
        
btn_load.on_click(on_click_load)


btn_diff = Button(description='Show Difference')
out_pl_diff = Output()

def on_click_find_diff(change):
    imReference = np.array(Image.open(io.BytesIO(btn1_upload.data[-1])))
    im = np.array(Image.open(io.BytesIO(btn2_upload.data[-1])))
    
    kernel = np.ones((6,6),np.uint8)
    blur_kern = (3,3)
    
    imReference = cv2.blur(imReference, blur_kern)
    imReference = cv2.morphologyEx(imReference, cv2.MORPH_OPEN, kernel)
    
    im = cv2.blur(im, blur_kern)
    im = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel)

    imReg = alignImages(im, imReference)
    
#     remove black borders from aligned image
    black_border = np.where(imReg==0)
    imRef_crop = imReference.copy()
    imRef_crop[black_border]=0

#     find difference between images
    diff = cv2.absdiff(imReg, imRef_crop)
        
    out_pl_diff.clear_output()
    with out_pl_diff:
        result = (diff > 75) * diff
        img2 = Image.fromarray(result)                
        img2.thumbnail([450,450])
        display(img2.rotate(rotation))
            
btn_diff.on_click(on_click_find_diff)

In [4]:
VBox([HBox([btn1_upload, btn2_upload, btn_load, btn_diff]), HBox([out_pl_ref, out_pl_im, out_pl_diff])])

In [5]:
import session_info

In [6]:
session_info.show()